# Documentation for Homomorphic Distance and Similarity Computation

## Overview
This program demonstrates the use of homomorphic encryption for secure computation of **Euclidean distance** and **cosine similarity** between two vectors. The implementation uses the **TenSEAL** library to encrypt vectors and perform operations on encrypted data without decrypting it. The results are then decrypted and compared with computations on cleartext data to evaluate accuracy and runtime performance.

---

## Key Components

### Libraries Used
- **TenSEAL**: For homomorphic encryption and encrypted vector computations.
- **NumPy**: For vector operations in cleartext.
- **time**: For performance measurement.
- **statistics**: For calculating accuracy and runtime statistics.

### Encryption Scheme
The **CKKS** scheme (Cheon-Kim-Kim-Song) is used, which supports approximate arithmetic on encrypted data. The context parameters include:
- **Polynomial modulus degree**: 8192 (controls computation capacity).
- **Coefficient modulus bit sizes**: `[60, 40, 40, 60]` (defines the scale and precision).
- **Global scale**: $$2^{40}$$ (precision for encoded numbers).

---

## Functions

### Homomorphic Euclidean Distance
The Euclidean distance between two encrypted vectors is computed as follows:
1. Compute the element-wise difference: $$\text{diff} = \text{vec1} - \text{vec2}$$.
2. Square the differences: $$\text{squared\_diff} = \text{diff}^2$$.
3. Sum the squared differences: $$\text{sum\_squared\_diff} = \sum \text{squared\_diff}$$.
4. Decrypt and take the square root to get the distance:
   $$
   \text{distance} = \sqrt{\text{sum\_squared\_diff}}
   $$

### Homomorphic Cosine Similarity
The cosine similarity between two encrypted vectors is computed as:
1. Compute the dot product: $$\text{dot\_product} = \sum (\text{vec1} \times \text{vec2})$$.
2. Compute the norms of the vectors:
   $$
   \text{norm1} = \sum (\text{vec1}^2), \quad \text{norm2} = \sum (\text{vec2}^2)
   $$
3. Decrypt the results and calculate:
   $$
   \text{cosine similarity} = \frac{\text{dot\_product}}{\sqrt{\text{norm1}} \times \sqrt{\text{norm2}}}
   $$

---

## Experimental Setup

### Parameters
- **Vector Dimension**: 512
- **Repetitions**: 100 trials for accuracy and runtime measurement.

### Process
1. **Vector Generation**:
   - Two random vectors are generated using uniform distribution \([0.0, 1.0]\).
2. **Encryption**:
   - The vectors are encrypted using CKKS.
3. **Computation**:
   - Encrypted Euclidean distance and cosine similarity are computed.
4. **Decryption and Comparison**:
   - Results are decrypted and compared with cleartext computations.
5. **Runtime Measurements**:
   - Times for vector generation, encryption, computation, and decryption are recorded.

---

## Results

### Accuracy
Accuracy is evaluated as the absolute difference between encrypted and cleartext results for both metrics:
- **Euclidean Distance**:
  $$
  \text{Error} = |\text{Encrypted Euclidean} - \text{Cleartext Euclidean}|
  $$
  - **Average Error**: $$6.561366 \times 10^{-7}$$
  - **Standard Deviation**: $$1.580893 \times 10^{-8}$$
  - **Maximum Error**: $$6.838830 \times 10^{-7}$$

- **Cosine Similarity**:
  $$
  \text{Error} = |\text{Encrypted Cosine} - \text{Cleartext Cosine}|
  $$
  - **Average Error**: $$1.116955 \times 10^{-9}$$
  - **Standard Deviation**: $$7.678165 \times 10^{-10}$$
  - **Maximum Error**: $$3.488762 \times 10^{-9}$$

### Runtime
Runtime is measured for the following steps:
- **Vector Generation**:
  - **Average**: $$0.0000 \text{s}$$
  - **Standard Deviation**: $$0.0002 \text{s}$$
  - **Maximum**: $$0.0010 \text{s}$$

- **Encryption**:
  - **Average**: $$0.0149 \text{s}$$
  - **Standard Deviation**: $$0.0026 \text{s}$$
  - **Maximum**: $$0.0335 \text{s}$$

- **Computation**:
  - **Average**: $$0.1202 \text{s}$$
  - **Standard Deviation**: $$0.0096 \text{s}$$
  - **Maximum**: $$0.1399 \text{s}$$

- **Decryption**:
  - **Average**: $$0.0001 \text{s}$$
  - **Standard Deviation**: $$0.0003 \text{s}$$
  - **Maximum**: $$0.0015 \text{s}$$

---

## Final Demonstration
In the final demo:
1. Two new vectors are generated and encrypted.
2. Both metrics are computed homomorphically and decrypted.
3. Results are compared with cleartext computations:
   - **Euclidean Distance** (Encrypted vs. Cleartext): $$9.046561 \text{ vs. } 9.046561$$
   - **Cosine Similarity** (Encrypted vs. Cleartext): $$0.745666 \text{ vs. } 0.745666$$

---

## Future Directions
1. Optimize parameters for CKKS to balance precision and performance.
2. Scale up to larger vector dimensions or batch processing.
3. Test on real-world encrypted data to evaluate robustness.

---


In [1]:
import time
import statistics
import numpy as np
import tenseal as ts

VECTOR_DIM = 512   

REPETITIONS = 100

context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()

context.global_scale = 2 ** 40

def homomorphic_euclidean_distance(enc_vec1, enc_vec2):
    diff = enc_vec1 - enc_vec2
    squared_diff = diff * diff
    sum_squared_diff = squared_diff.sum()
    decrypted_sum = sum_squared_diff.decrypt()[0]
    return np.sqrt(decrypted_sum)

def homomorphic_cosine_similarity(enc_vec1, enc_vec2):
    dot_product = (enc_vec1 * enc_vec2).sum()
    norm1 = (enc_vec1 * enc_vec1).sum()
    norm2 = (enc_vec2 * enc_vec2).sum()

    decrypted_dot = dot_product.decrypt()[0]
    decrypted_norm1 = norm1.decrypt()[0]
    decrypted_norm2 = norm2.decrypt()[0]

    return decrypted_dot / (np.sqrt(decrypted_norm1) * np.sqrt(decrypted_norm2))

accuracy_results = {"Euclidean": [], "Cosine": []}

runtime_results = {
    "Generation": [],
    "Encryption": [],
    "Computation": [],
    "Decryption": []
}

for _ in range(REPETITIONS):
    t0 = time.time()
    vector1 = np.random.uniform(0.0, 1.0, size=VECTOR_DIM)
    vector2 = np.random.uniform(0.0, 1.0, size=VECTOR_DIM)
    runtime_results["Generation"].append(time.time() - t0)

    t0 = time.time()
    enc_vec1 = ts.ckks_vector(context, vector1)
    enc_vec2 = ts.ckks_vector(context, vector2)
    runtime_results["Encryption"].append(time.time() - t0)

    t0 = time.time()
    encrypted_euclidean = homomorphic_euclidean_distance(enc_vec1, enc_vec2)
    encrypted_cosine = homomorphic_cosine_similarity(enc_vec1, enc_vec2)
    runtime_results["Computation"].append(time.time() - t0)

    t0 = time.time()
    cleartext_euclidean = np.linalg.norm(vector1 - vector2)
    cleartext_cosine = np.dot(vector1, vector2) / (
        np.linalg.norm(vector1) * np.linalg.norm(vector2)
    )
    accuracy_results["Euclidean"].append(abs(encrypted_euclidean - cleartext_euclidean))
    accuracy_results["Cosine"].append(abs(encrypted_cosine - cleartext_cosine))
    runtime_results["Decryption"].append(time.time() - t0)

import math
import statistics

print("\n=== Accuracy Results (Absolute Differences) ===")
for metric, values in accuracy_results.items():
    avg_diff = statistics.mean(values)
    std_diff = statistics.pstdev(values) if len(values) > 1 else 0.0
    max_diff = max(values)
    print(f"{metric}: avg={avg_diff:.6e}, std={std_diff:.6e}, max={max_diff:.6e}")

print("\n=== Runtime Results (seconds) ===")
for step, times in runtime_results.items():
    avg_t = statistics.mean(times)
    std_t = statistics.pstdev(times) if len(times) > 1 else 0.0
    max_t = max(times)
    print(f"{step}: avg={avg_t:.4f}s, std={std_t:.4f}s, max={max_t:.4f}s")

print("\n=== Final Demo ===")
demo_vec1 = np.random.uniform(0.0, 1.0, size=VECTOR_DIM)
demo_vec2 = np.random.uniform(0.0, 1.0, size=VECTOR_DIM)
ct_demo_vec1 = ts.ckks_vector(context, demo_vec1)
ct_demo_vec2 = ts.ckks_vector(context, demo_vec2)

encrypted_euc_demo = homomorphic_euclidean_distance(ct_demo_vec1, ct_demo_vec2)
encrypted_cos_demo = homomorphic_cosine_similarity(ct_demo_vec1, ct_demo_vec2)

clear_euc_demo = np.linalg.norm(demo_vec1 - demo_vec2)
clear_cos_demo = np.dot(demo_vec1, demo_vec2) / (
    np.linalg.norm(demo_vec1) * np.linalg.norm(demo_vec2)
)

print(f"Euclidean distance (encrypted) = {encrypted_euc_demo:.6f} vs. cleartext = {clear_euc_demo:.6f}")
print(f"Cosine similarity (encrypted) = {encrypted_cos_demo:.6f} vs. cleartext = {clear_cos_demo:.6f}")



=== Accuracy Results (Absolute Differences) ===
Euclidean: avg=6.561366e-07, std=1.580893e-08, max=6.838830e-07
Cosine: avg=1.116955e-09, std=7.678165e-10, max=3.488762e-09

=== Runtime Results (seconds) ===
Generation: avg=0.0000s, std=0.0002s, max=0.0010s
Encryption: avg=0.0149s, std=0.0026s, max=0.0335s
Computation: avg=0.1202s, std=0.0096s, max=0.1399s
Decryption: avg=0.0001s, std=0.0003s, max=0.0015s

=== Final Demo ===
Euclidean distance (encrypted) = 9.046561 vs. cleartext = 9.046561
Cosine similarity (encrypted) = 0.745666 vs. cleartext = 0.745666
